In [46]:
import numpy as np

L= 4
n=[4,8,8,8,1]

#creating the weight matrix and bias matrices

W1=np.random.randn(n[1],n[0])
W2=np.random.randn(n[2],n[1])
W3=np.random.randn(n[3],n[2])
W4=np.random.randn(n[4],n[3])

b1=np.random.randn(n[1],1)
b2=np.random.randn(n[2],1)
b3=np.random.randn(n[3],1)
b4=np.random.randn(n[4],1)

In [47]:
def my_data():
    X = np.array([#input array with binary values of 10 random numbers in 4-bit notation
    [0,0,1,1],
    [1,1,0,0],
    [0,1,1,1],
    [0,0,0,1],
    [1,1,1,0],
    [0,1,0,1],
    [1,0,0,1],
    [0,0,1,0],
    [1,0,1,1],
    [1,0,1,1]])
    
    Yi=np.array([3, 12, 7, 1, 14, 5, 9, 2, 11, 0])/15
# scaling the output by 15 since 15 is the highest 4-bit number, scaling it to limit the values between 0 and 1 to later convert it to numerical values
    m=10
    A0=X.T
    Y=Yi.reshape(n[L],m)
    
    return A0,Y

In [48]:
# def cost(y_hat,y):
#     loss= -(y*np.log(y_hat) + (1-y)*np.log(1-y_hat))
    
#     m=y_hat.reshape(-1).shape[0]
    
#     sum_of_losses= (1/m)* np.sum(loss,axis=1)
    
#     return sum_of_losses

def cost(y_hat, y):
    m = y.shape[1]
    loss = (1/m) * np.sum((y_hat - y)**2)
    return loss


In [49]:
def sigmoid_function(x):
    return 1/(1+np.exp(-1*x))



In [50]:
def feed_forward(A0):
    
    z1=W1@A0+b1
    a1=sigmoid_function(z1)
    
    z2=W2@a1+b2
    a2=sigmoid_function(z2)
    
    z3=W3@a2+b3
    a3=sigmoid_function(z3)
    
    z4=W4@a3+b4
    a4=sigmoid_function(z4)
    
    cache = {
        "A0": A0,
        "A1": a1,
        "A2": a2,
        "A3": a3,
        "A4": a4
    } #the cache values are needed for backpropagation
    
    
    return a4, cache

In [51]:
def backprop_l_4(y_hat,y,m,a3,w4):
    
    a4 = y_hat
    dC_dz4 = (1/m)*(a4-y)
    assert dC_dz4.shape == (n[4], m)
    
    # dc/dw4 = dc/dz4 * dz4/dw4
    
    dz4_dw4 = a3
    assert dz4_dw4.shape == (n[3], m)
    
    dc_dw4 = dC_dz4 @ dz4_dw4.T
    assert dc_dw4.shape == (n[4], n[3])
    
    
    # dc/db4 = dc/dz4 * dz4/db4
    dc_db4 = np.sum(dC_dz4, axis=1, keepdims=True)
    assert dc_db4.shape == (n[4], 1)
    
    # calculating the propagator dc/da3
    dz4_da3 = w4
    dc_da3 = w4.T @dC_dz4
    assert dc_da3.shape == (n[3], m)
    
    return dc_dw4, dc_db4, dc_da3


def backprop_l_3(propagator_dc_da3, a2,a3, w3, m):
    
    # dc/dz3 = dc/da3 * da3/dz3
    
    da3_dz3 = a3*(1-a3)
    dc_dz3 = propagator_dc_da3 * da3_dz3
    assert dc_dz3.shape == (n[3], m)
    
    # dc/dw2 = dc/dz3 * dz3/dw3
    
    dz3_dw3 = a2
    assert dz3_dw3.shape == (n[2], m)
    
    dc_dw3 = dc_dz3 @ dz3_dw3.T
    assert dc_dw3.shape == (n[3], n[2])
    
    dc_db3 = np.sum(dc_dz3, axis=1, keepdims=True)
    assert dc_db3.shape == (n[3], 1)
    
    # calculating the propagator dc/da2
    dz3_da2 = w3
    dc_da2 = w3.T @ dc_dz3
    assert dc_da2.shape == (n[2], m)
    
    return dc_dw3, dc_db3, dc_da2

def backprop_l_2(propagator_dc_da2, a2,a1, w2, m):
    # dc/dz2 = dc/da2 * da2/dz2
    
    da2_dz2 = a2*(1-a2)
    dc_dz2 = propagator_dc_da2 * da2_dz2
    assert dc_dz2.shape == (n[2], m)
    
    # dc/dw2 = dc/dz2 * dz2/dw2
    
    dz2_dw2 = a1
    assert dz2_dw2.shape == (n[1], m)
    
    dc_dw2 = dc_dz2 @ dz2_dw2.T
    assert dc_dw2.shape == (n[2], n[1])
    
    dc_db2 = np.sum(dc_dz2, axis=1, keepdims=True)
    assert dc_db2.shape == (n[2], 1)
    
    # calculating the propagator dc/da1
    dz2_da1 = w2
    dc_da1 = w2.T @ dc_dz2
    assert dc_da1.shape == (n[1], m)
    
    return dc_dw2, dc_db2, dc_da1



def backprop_l_1(propagator_dc_da1, a1,A0, w1, m):
    # dc/dz1 = dc/da1 * da1/dz1
    
    da1_dz1 = a1*(1-a1)
    dc_dz1 = propagator_dc_da1 * da1_dz1
    assert dc_dz1.shape == (n[1], m)
    
    # dc/dw1 = dc/dz1 * dz1/dw1
    
    dz1_dw1 = A0
    assert dz1_dw1.shape == (n[0], m)
    
    dc_dw1 = dc_dz1 @ dz1_dw1.T
    assert dc_dw1.shape == (n[1], n[0])
    
    dc_db1 = np.sum(dc_dz1, axis=1, keepdims=True)
    assert dc_db1.shape == (n[1], 1)
    
    # calculating the propagator dc/da0
    dz1_da0 = w1
    dc_da0 = w1.T @ dc_dz1
    assert dc_da0.shape == (n[0], m)
    
    return dc_dw1, dc_db1, dc_da0

In [56]:
def train():
    global W1, W2, W3, W4, b1, b2, b3, b4
    
    epochs = 10000
    alpha = 0.1 # learning rate
    costs = []
    
    for e in range(epochs):
        # feed forward
        y_hat,cache = feed_forward(A0)
        
        # cost calculation
        error = cost(y_hat, Y)
        costs.append(error)
        
        # backpropagation
        dC_dw4,dc_db4,dc_da3 = backprop_l_4(y_hat, Y, m, cache["A3"], w4=W4)
        
        dC_dw3,dc_db3,dc_da2 = backprop_l_3(dc_da3, a2= cache["A2"],a3=cache["A3"], w3=W3,m=m)
        
        dC_dw2,dc_db2,dc_da1 = backprop_l_2(dc_da2, a1=cache["A1"],a2=cache["A2"], w2=W2,m=m)
        
        dC_dw1,dc_db1,dc_da0 = backprop_l_1(dc_da1, a1= cache["A1"],A0=cache["A0"] , w1=W1,m=m)
        
        
        # update_weights
        
        W4 -= alpha * dC_dw4
        W3 -= alpha * dC_dw3
        W2 -= alpha * dC_dw2    
        W1 -= alpha * dC_dw1
        
        b4 -= alpha * dc_db4
        b3 -= alpha * dc_db3    
        b2 -= alpha * dc_db2
        b1 -= alpha * dc_db1    
        if e % 10 == 0:
            print(f"epoch {e}: cost = {error.mean():.6f}")
    return costs

In [57]:
A0, Y = my_data()  # Get input and target
m = A0.shape[1]    # Update m here if not global
costs = train()    # Actually train the model!

epoch 0: cost = 0.027821
epoch 10: cost = 0.027819
epoch 20: cost = 0.027817
epoch 30: cost = 0.027815
epoch 40: cost = 0.027813
epoch 50: cost = 0.027811
epoch 60: cost = 0.027809
epoch 70: cost = 0.027807
epoch 80: cost = 0.027805
epoch 90: cost = 0.027803
epoch 100: cost = 0.027801
epoch 110: cost = 0.027799
epoch 120: cost = 0.027797
epoch 130: cost = 0.027795
epoch 140: cost = 0.027793
epoch 150: cost = 0.027791
epoch 160: cost = 0.027789
epoch 170: cost = 0.027787
epoch 180: cost = 0.027785
epoch 190: cost = 0.027783
epoch 200: cost = 0.027781
epoch 210: cost = 0.027779
epoch 220: cost = 0.027777
epoch 230: cost = 0.027775
epoch 240: cost = 0.027774
epoch 250: cost = 0.027772
epoch 260: cost = 0.027770
epoch 270: cost = 0.027768
epoch 280: cost = 0.027766
epoch 290: cost = 0.027764
epoch 300: cost = 0.027762
epoch 310: cost = 0.027760
epoch 320: cost = 0.027758
epoch 330: cost = 0.027756
epoch 340: cost = 0.027754
epoch 350: cost = 0.027753
epoch 360: cost = 0.027751
epoch 370: c

In [58]:
# After training
y_hat, _ = feed_forward(A0)  # Forward pass with trained weights

# Convert back to decimal (0–15)
predicted_values = np.round(y_hat * 15).astype(int)  # shape: (1, m)
actual_values = np.round(Y * 15).astype(int)         # shape: (1, m)

# Print predictions
print("Predictions:")
print(predicted_values.flatten())
print("Actual:")
print(actual_values.flatten())

# Calculate accuracy
correct = (predicted_values == actual_values).sum()
total = predicted_values.size
accuracy = (correct / total) * 100
print(f"Accuracy: {accuracy:.2f}%")

# Optional: Mean Absolute Error
mae = np.mean(np.abs(predicted_values - actual_values))
print(f"Mean Absolute Error: {mae:.2f}")


Predictions:
[ 3 12  7  1 14  5  9  2  6  6]
Actual:
[ 3 12  7  1 14  5  9  2 11  0]
Accuracy: 80.00%
Mean Absolute Error: 1.10
